# 10 — Geração de Submissão para o Kaggle

## Objetivo

Este notebook tem como finalidade:

1. Carregar o modelo treinado com melhor desempenho.
2. Realizar inferência no conjunto de teste privado.
3. Gerar o arquivo `.csv` no formato exigido pela competição.

---

## Métrica Oficial

A trilha de Visão Computacional utiliza **ROC-AUC** como métrica oficial.  
Portanto, o arquivo de submissão deve conter **as probabilidades da classe positiva (Pneumonia)**, e não rótulos binários.

Formato exigido:

id,target  
img_0001.jpeg,0.95  
img_0002.jpeg,0.02  

---

## Reprodutibilidade

Este notebook:
- Não treina modelos.
- Não altera pesos.
- Apenas carrega o modelo salvo.
- Aplica transformações determinísticas.
- Gera submissão consistente.

Para alterar o modelo utilizado, basta modificar a variável `MODEL_FILENAME`.

In [10]:
import os
import sys
import torch
import pandas as pd
import numpy as np
from tqdm import tqdm
from PIL import Image

# ===============================
# Definição de caminhos do projeto
# ===============================

PROJECT_ROOT = os.path.abspath(os.path.join(os.getcwd(), ".."))
SRC_PATH = os.path.join(PROJECT_ROOT, "src")
DATA_PATH = os.path.join(PROJECT_ROOT, "data")
MODELS_PATH = os.path.join(PROJECT_ROOT, "models")
OUTPUTS_PATH = os.path.join(PROJECT_ROOT, "outputs")
SUBMISSIONS_PATH = os.path.join(OUTPUTS_PATH, "submissions")

if SRC_PATH not in sys.path:
    sys.path.append(SRC_PATH)

print("Project root:", PROJECT_ROOT)

from model import get_model
from transforms import get_transforms

Project root: c:\projects\xray-project


## Seleção do Modelo

Para gerar a submissão, selecione:

- A arquitetura (`MODEL_NAME`)
- O arquivo de pesos salvo (`MODEL_FILENAME`)

Após executar todos os experimentos, basta alterar essas variáveis
para gerar submissão com o melhor modelo.

In [ ]:
# escolher manualmente o modelo que a pessoa quer o .csv

MODEL_NAME = "resnet18"
MODEL_FILENAME = "resnet18_strong_noCW.pt"  # Melhor modelo
IMG_SIZE = 224

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

print("Modelo selecionado:", MODEL_FILENAME)
print("Device:", DEVICE)

Modelo selecionado: resnet18_strong_noCW.pt
Device: cpu


## Carregamento do Modelo Treinado

O modelo é carregado a partir da pasta `models/`.

Utilizamos `model.eval()` para garantir que:
- BatchNorm opere em modo de avaliação.
- Dropout seja desativado.

In [21]:
# Instanciar arquitetura
model, _ = get_model(model_name=MODEL_NAME)

# Carregar pesos treinados
model_path = os.path.join(MODELS_PATH, MODEL_FILENAME)
model.load_state_dict(torch.load(model_path, map_location=DEVICE))

model.to(DEVICE)
model.eval()

print("Modelo carregado com sucesso.")

c:\Users\honor\AppData\Local\Programs\Python\Python310\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\honor\AppData\Local\Programs\Python\Python310\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Modelo carregado com sucesso.


## Transformações de Teste

Durante a inferência:

- NÃO aplicamos data augmentation.
- Apenas resize e normalização.
- Garantimos consistência com o pipeline de validação.

In [22]:
# Para inferência usamos transform de validação
_, test_transform = get_transforms(img_size=IMG_SIZE, augmentation="light")

print("Transformação de teste definida.")

Transformação de teste definida.


In [23]:
# Carregar a lista de teste 

test_csv_path = os.path.join(DATA_PATH, "test.csv")
test_images_dir = os.path.join(DATA_PATH, "test_images")

test_df = pd.read_csv(test_csv_path)

print("Número de imagens de teste:", len(test_df))
test_df.head()

Número de imagens de teste: 624


,id
0,img_0001.jpeg
1,img_0002.jpeg
2,img_0003.jpeg
3,img_0004.jpeg
4,img_0005.jpeg


## Processo de Inferência

Para cada imagem:
1. Carregamos a imagem.
2. Aplicamos transformação.
3. Realizamos forward pass.
4. Extraímos a probabilidade da classe Pneumonia.

In [24]:
all_ids = []
all_probs = []

for _, row in tqdm(test_df.iterrows(), total=len(test_df)):
    
    img_name = row["id"]
    img_path = os.path.join(test_images_dir, img_name)
    
    image = Image.open(img_path).convert("RGB")
    image = test_transform(image)
    image = image.unsqueeze(0).to(DEVICE)
    
    with torch.no_grad():
        output = model(image)
        prob = torch.softmax(output, dim=1)[0, 1].item()
    
    all_ids.append(img_name)
    all_probs.append(prob)

100%|██████████| 624/624 [00:44<00:00, 14.15it/s]


## Construção do CSV

O arquivo final contém:

- `id` → nome da imagem
- `target` → probabilidade da classe Pneumonia

In [25]:
submission_df = pd.DataFrame({
    "id": all_ids,
    "target": all_probs
})

submission_df.head()

,id,target
0,img_0001.jpeg,0.010095
1,img_0002.jpeg,0.154185
2,img_0003.jpeg,0.999966
3,img_0004.jpeg,0.006217
4,img_0005.jpeg,0.012442


In [26]:
# Nome automático baseado no modelo
submission_filename = f"submission_{MODEL_FILENAME.replace('.pt','')}.csv"

submission_path = os.path.join(SUBMISSIONS_PATH, submission_filename)

submission_df.to_csv(submission_path, index=False)

print("Arquivo de submissão salvo em:")
print(submission_path)

Arquivo de submissão salvo em:
c:\projects\xray-project\outputs\submissions\submission_resnet18_strong_noCW.csv


## Verificação das Probabilidades

Garantimos que todas as probabilidades estejam no intervalo [0,1].

In [27]:
print("Probabilidade mínima:", min(all_probs))
print("Probabilidade máxima:", max(all_probs))

Probabilidade mínima: 1.453556478736573e-06
Probabilidade máxima: 1.0
